In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
%matplotlib inline

In [2]:
# Create a equally balanced portoflio of FB, AMZN, AAPL, NFLX, GOOGL
# Rebalance weekly, with excess in cash 
# Measure performance from 2018-01-01 until today

In [3]:
# Gather Data
start = '2018-01-01'
end = '2019-04-01'
FB = web.DataReader('FB', 'iex',start, end)
AMZN = web.DataReader('AMZN', 'iex',start, end)
AAPL = web.DataReader('AAPL', 'iex',start, end)
NFLX = web.DataReader('NFLX', 'iex',start, end)
GOOGL = web.DataReader('GOOGL', 'iex',start, end)

In [4]:
# Add daily return column
for stock_df in (FB, AMZN, AAPL, NFLX, GOOGL):
    stock_df['Daily Returns'] = stock_df['close'].pct_change(1)

In [5]:
# Add cumulative returns 
for stock_df in (FB, AMZN, AAPL, NFLX, GOOGL):
    stock_df['Cumulative Returns'] = stock_df['close']/stock_df['close'].iloc[0]

In [6]:
# Simulate a starting portfolio
start_value = 1000000
equal_weight = 0.2
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL','Cash']
d = {'Target':[0.2, 0.2, 0.2, 0.2, 0.2, 0],
    'Date': pd.to_datetime('2018-1-1'),
    'Upper': 0.25,
    'Lower': 0.15}
start_port = pd.DataFrame(index=tickers,data=d)
start_port

,Target,Date,Upper,Lower
FB,0.2,2018-01-01,0.25,0.15
AMZN,0.2,2018-01-01,0.25,0.15
AAPL,0.2,2018-01-01,0.25,0.15
NFLX,0.2,2018-01-01,0.25,0.15
GOOGL,0.2,2018-01-01,0.25,0.15
Cash,0.0,2018-01-01,0.25,0.15


In [7]:
# Set the intial positions for the portfolio 
cash = 0

fb_shares_start = (start_value * equal_weight) // FB['close'][0]
cash += (start_value * equal_weight) % FB['close'][0]

amzn_shares_start = (start_value * equal_weight) // AMZN['close'][0]
cash += (start_value * equal_weight) % AMZN['close'][0]


aapl_shares_start = (start_value * equal_weight) // AAPL['close'][0]
cash += (start_value * equal_weight) % AAPL['close'][0]

nflx_shares_start = (start_value * equal_weight) // NFLX['close'][0]
cash += (start_value * equal_weight) % NFLX['close'][0]

googl_shares_start = (start_value * equal_weight) // GOOGL['close'][0]
cash += (start_value * equal_weight) % GOOGL['close'][0]

start_positions = [fb_shares_start, amzn_shares_start, aapl_shares_start, 
                                 nflx_shares_start, googl_shares_start, 0]

start_positions

[1102.0, 168.0, 1183.0, 994.0, 186.0, 0]

In [8]:
start_port['Shares'] = start_positions
start_port

,Target,Date,Upper,Lower,Shares
FB,0.2,2018-01-01,0.25,0.15,1102.0
AMZN,0.2,2018-01-01,0.25,0.15,168.0
AAPL,0.2,2018-01-01,0.25,0.15,1183.0
NFLX,0.2,2018-01-01,0.25,0.15,994.0
GOOGL,0.2,2018-01-01,0.25,0.15,186.0
Cash,0.0,2018-01-01,0.25,0.15,0.0


In [9]:
# Market Value per position
fb_mv = fb_shares_start * FB['close'][0]
amzn_mv = amzn_shares_start * AMZN['close'][0]
aapl_mv = aapl_shares_start * AAPL['close'][0]
nflx_mv = nflx_shares_start * NFLX['close'][0]
googl_mv = googl_shares_start * GOOGL['close'][0]

start_mv = [fb_mv, amzn_mv, aapl_mv, nflx_mv, googl_mv, cash]

start_port['Market Value'] = start_mv
start_port = start_port[['Date', 'Target', 'Upper', 'Lower', 'Shares', 'Market Value']]
start_port

,Date,Target,Upper,Lower,Shares,Market Value
FB,2018-01-01,0.2,0.25,0.15,1102.0,199924.8400
AMZN,2018-01-01,0.2,0.25,0.15,168.0,199753.6800
AAPL,2018-01-01,0.2,0.25,0.15,1183.0,199911.8576
NFLX,2018-01-01,0.2,0.25,0.15,994.0,199863.5800
GOOGL,2018-01-01,0.2,0.25,0.15,186.0,199617.0600
Cash,2018-01-01,0.0,0.25,0.15,0.0,928.9824


In [19]:
tickers = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOGL','Cash']
allocations = []
for tick in tickers:
    allocations.append(start_port.loc[tick]['Market Value']/ start_port['Market Value'].sum())

start_port['Allocations']=allocations
start_port

,Date,Target,Upper,Lower,Shares,Market Value,Allocations
FB,2018-01-01,0.2,0.25,0.15,11024.0,1999974.080,0.199997
AMZN,2018-01-01,0.2,0.25,0.15,1682.0,1999914.820,0.199991
AAPL,2018-01-01,0.2,0.25,0.15,11835.0,1999963.512,0.199996
NFLX,2018-01-01,0.2,0.25,0.15,9946.0,1999842.220,0.199984
GOOGL,2018-01-01,0.2,0.25,0.15,1863.0,1999390.230,0.199939
Cash,2018-01-01,0.0,0.00,0.00,0.0,915.138,0.000092


In [20]:
FB.head()

,open,high,low,close,volume,Daily Returns,Cumulative Returns
date,,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,NaN,1.000000
2018-01-03,181.88,184.78,181.3300,184.67,16886563,0.017914,1.017914
2018-01-04,184.90,186.21,184.0996,184.33,13880896,-0.001841,1.016040
2018-01-05,185.59,186.90,184.9300,186.85,13574535,0.013671,1.029931
2018-01-08,187.20,188.90,186.3300,188.28,17994726,0.007653,1.037813
